# Elliptic Curve Diffie-Hellman

Протокол для обмена общим секретом.

Сетап:
- $E(F_p)$
- $G \in E(F_p)$

Алиса и боб оба генерируют по случайному секретному числу:

$a, b \in [1, ord(G)-1]$

Алиса отсылает Бобу $G_a = a * G$ 

Боб отсылает Алисе $G_b = b * G$

\begin{rcases}
   a * G_b &\text{Алиса }  \\
   b * G_a &\text{Боб } 
\end{rcases} => $(a * b) * G = (b * a) * G = Sh$

In [1]:
import hashlib 

class Part:
    def __init__(self, E, G):
        self.private = randint(1, G.order()-1)
        self.E = E
        self.G = G

    def compute_public(self):
        return self.private * G

    def compute_shared(self, G_b):
        self.shared = self.private * G_b
        return self.shared

    def derive_key(self):
        self.key = hashlib.blake2b(str(self.shared).encode(), digest_size=16).digest()

    def encrypt_message(self, m: bytes):
        return bytes([x^^y for x, y in zip(m, self.key * ((len(m) + 15) // 16))])

    def decrypt_message(self, m: bytes):
        return self.encrypt_message(m).decode()
        
        
p = random_prime(2**255)
a = randint(0, p-1)
b = randint(0, p-1)
E = EllipticCurve(GF(p), [a, b])
G = E.random_element()

A = Part(E, G)
B = Part(E, G)

A_pub = A.compute_public()
B_pub = B.compute_public()

Shared_A = A.compute_shared(B_pub)
Shared_B = B.compute_shared(A_pub)
assert Shared_A == Shared_B

# Man in the middle



In [37]:
class C:
    def __init__(self, E, G):
        pass
    ...

A = Part(E, G)
B = Part(E, G)

A_pub = A.compute_public()
B_pub = B.compute_public()

Eve = C(E, G)

# do smth wiith A and B
A.compute_shared(B_pub)
B.compute_shared(A_pub)


A.derive_key()
ct = A.encrypt_message(b"Send me Hello together with our secret code")
# do smth here

B.derive_key()
m = B.decrypt_message(ct)
ct = B.encrypt_message(b"Hello, my lil potato")
# do smth here

m = A.decrypt_message(ct)
assert 'Hello' in m and 'lil potato' in m

# Гладкий порядок кривой и элемента

Допустим у нас имеется точка кривой с порядком $ord(G) = p * q$. Что произойдёт, если умножить эту точку на $p$?

In [4]:
q = 21888242871839275222246405745257275088696311157297823662689037894645226208583
E = EllipticCurve(GF(q), [0, 1])

print(factor(E.order()))

2^2 * 3^3 * 13^2 * 19 * 29^2 * 37 * 613 * 983^2 * 11003^2 * 346501 * 6248149 * 405928799^2 * 79287328374952431757


In [8]:
P = E.gens()[0]
print(factor(P.order()))
    
n = 79287328374952431757 * randint(0, 27797133921898830561267529521791838546)
Q = n * P
print(factor(P.order()))
print(factor(Q.order()))

2 * 3^2 * 13 * 19 * 29 * 37 * 613 * 983 * 11003 * 346501 * 6248149 * 405928799 * 79287328374952431757
2 * 3^2 * 13 * 19 * 29 * 37 * 613 * 983 * 11003 * 346501 * 6248149 * 405928799 * 79287328374952431757
2 * 3^2 * 13 * 29 * 37 * 613 * 983 * 11003 * 346501 * 6248149 * 405928799


In [9]:
def bsgs(P, Q, n):
    if Q == Q.curve()((0, 1, 0)):
        return 0

    if n < 17:
        for i in range(n):
            if P * i == Q:
                return i
        
    m = floor(sqrt(n))
    precomp = {}
    P0 = P.curve()((0, 1, 0))
    
    for j in range(m):
        precomp[P0[0]] = j
        P0 += P
    
    mP = m * P
    Q0 = Q
    for i in range(m):
        if Q0[0] in precomp:
            t1 = i * m + precomp[Q0[0]]
            t2 = i * m - precomp[Q0[0]]
            if t1 * P == Q:
                return t1 % n
            return t2 % n
        Q0 -= mP
    return None

# Небольшое напоминание CRT

Вы имеете систему уравнений:

$x \equiv a_1 \pmod{p_1^{e_1}}$

$x \equiv a_2 \pmod{p_2^{e_2}}$

...

$x \equiv a_n \pmod{p_n^{e_n}}$

(Есть и более общий случай для непростых-степеней модулей но об этом не сегодня)

Тогда по Китайской Теореме об Остатках существует единственное решение этой системы:

$x = a^{*} \pmod{p_1^{e_1} * ... * p_n^{e_n}}$

В саге это можно посчитать: $x = crt([a_1, a_2, ..., a_n], [p_1^{e_1}, p_2^{e_2}, ..., p_n^{e_n}]$

# Напоминаю про Полига-Хеллмана

$Q_1 = m * P_1, ord(P_1) = ord(Q_1) = p^e$

$m = m_0 + m_1 * p + ... + m_{e-1} * p^{e-1}$

$Q_1 = m_0 * P_1 + m_1 * (p * P_1) + ... + m_{e-1} * (p^{e - 1} * P)$

$p^{e - 1} * Q_1 = m_0 * (p^{e - 1} * P_1) + m_1 * (p^e * P_1) + m_2 * (p^{e + 1} * P_1) + ... + m_{e-1} * (p^{2 *e - 2} * P_1)$

$p^{e - 1} * Q_1 = m_0 * (p^{e - 1} * P_1) + \mathbb{O} + \mathbb{O} + ... + \mathbb{O} = m_0 * (p^{e - 1} * P_1), m_0 \in [0, p-1]$

После того как нашли $m_0$

$Q_1 = m_0 * P_1 + p * (m_1 *  P_1 + ... + m_{e-1} * (p^{e - 2} * P)$

$Q_1 - m_0 * P_1 = m_1 * (p * P_1) + m_2 * (p * (p * P_1)) + .. + m_{e - 1} * (p^{e - 2} * (p * P_1)$

$Q_2 = Q_1 - m_0 * P_1$

$P_2 = p * P_1$

$Q_2 = m_1 * P_2 + m_2 *(p * P_2) + .. + m_{e - 1} * (p^{e - 2} * P_2)$

$ord(P_2) = ord(Q_2) = p^{e -1 }$

In [10]:
o = P.order()
factors = factor(o)

ords, mods = [], []
for p, e in factors:
    print(f"{p, e =}")
    sc = o // (p^e)
    P1, Q1 = sc * P, sc * Q

    if Q1 == E((0, 1, 0)):
        print(f"m = 0")
        ords.append(0)
        mods.append(p**e)
        continue
        
    ms = []
    for i in range(e):
        Pi = P1 * p**(e - i - 1)
        Qi = Q1 * p**(e - i - 1)

        mi = bsgs(Pi, Qi, p)
        assert mi * Pi == Qi
        
        print(f"m_{i} = {mi}")
        ms.append(mi)
        
        Q1 = Q1 - mi * P1
        P1 = p * P1
        
    m = sum(p**i * c for i, c in enumerate(ms))
    print(f"{m = }")
    print("-" * 50)

    ords.append(m)
    mods.append(p**e)

n = crt(ords, mods)
print(n)
assert n * P == Q

p, e =(2, 1)
m_0 = 1
m = 1
--------------------------------------------------
p, e =(3, 2)
m_0 = 2
m_1 = 2
m = 8
--------------------------------------------------
p, e =(13, 1)
m_0 = 1
m = 1
--------------------------------------------------
p, e =(19, 1)
m = 0
p, e =(29, 1)
m_0 = 22
m = 22
--------------------------------------------------
p, e =(37, 1)
m_0 = 9
m = 9
--------------------------------------------------
p, e =(613, 1)
m_0 = 565
m = 565
--------------------------------------------------
p, e =(983, 1)
m_0 = 126
m = 126
--------------------------------------------------
p, e =(11003, 1)
m_0 = 8819
m = 8819
--------------------------------------------------
p, e =(346501, 1)
m_0 = 64811
m = 64811
--------------------------------------------------
p, e =(6248149, 1)
m_0 = 662381
m = 662381
--------------------------------------------------
p, e =(405928799, 1)
m_0 = 43977770
m = 43977770
--------------------------------------------------
p, e =(79287328374952431757, 1)
m = 

# Aномальные кривые

Из предыдущего пункта стало ясно, что лучше использовать либо точки с большим простым порядком, либо вообще использовать кривые с простым порядком, чтобы у всех точек был один и тот же большой простой порядок. 

Аномальными называют такие кривые у которых $|E(F_p)| = p$.

Для аномальных кривых существует отображение: $E(F_p) \to F_p: \phi(P)$, причем это отображение обладает очень важным свойством: $\phi(n * P) = n * \phi(P)$ - довольно печально выглядит не правда ли?

То есть от сложной проблемы мы перешли к обычному сложению.

Отображение считается следующим образом:

Введем пару $[P, a], [Q, b]$, тогда $[P, a] + [Q, b] = [P + Q, a + b + slope(P, Q)]$ 

Где
- $Q = -P, P = O$ или $Q = O \implies slope = 0$
- $slope = \lambda$

Тогда если мы посчитаем по данному правилу $p * [P, 0] = [P, 0] + [P, 0] + ... + [P, 0] = [O, \alpha]$, $\phi(P) = \alpha$

In [11]:
E = EllipticCurve(GF(101), [1, 69])
E.order()

101

In [12]:
p = 0xA15C4FB663A578D8B2496D3151A946119EE42695E18E13E90600192B1D0ABDBB6F787F90C8D102FF88E284DD4526F5F6B6C980BF88F1D0490714B67E8A2A2B77
a = 0x5E009506FCC7EFF573BC960D88638FE25E76A9B6C7CAEEA072A27DCD1FA46ABB15B7B6210CF90CABA982893EE2779669BAC06E267013486B22FF3E24ABAE2D42
b = 0x2CE7D1CA4493B0977F088F6D30D9241F8048FDEA112CC385B793BCE953998CAAE680864A7D3AA437EA3FFD1441CA3FB352B0B710BB3F053E980E503BE9A7FECE

E = EllipticCurve(GF(p), [a, b])
assert E.order() == p

P = E.random_element()
Q = randint(0, p) * P

In [13]:
def mul(P, n):
    if n < 0:
        P = -P
        n = -n
    b = bin(n)[2:]
    Q = P
    for i in b[1:]:
        Q = add(Q, Q)
        if i == "1":
            Q = add(Q, P)
    return Q


def a0(P, Q):
    if P.is_zero() or Q.is_zero() or P == -Q:
        return 0

    if P == Q:
        t = P.curve().a4()
        return (3 * P[0] ** 2 + t) * pow(2 * P[1], -1)

    return (P[1] - Q[1]) * pow(P[0] - Q[0], -1)


def add(P, Q):
    p1, scal1 = P
    p2, scal2 = Q
    p3 = p1 + p2
    scal3 = scal1 + scal2 + a0(p1, p2)
    return p3, scal3

In [14]:
G = GF(p)
P1 = (P, G(0))
Q1 = (Q, G(0))

R1, alpha = mul(P1, p)
R2, beta = mul(Q1, p)

res = beta / alpha
print(res)
assert res * P == Q

6580423973451703047762062349683967852989048968020199948724694186030410852629469815861811082547699222101664953208677803837179393306680320657110903129004768


In [15]:
def lift(A, B, P, Q, p):
    x1 = int(P[0]) + p * randint(0, p)
    x2 = int(Q[0]) + p * randint(0, p)
    y1 = int(P[1]) + p * randint(0, p)

    if x1 % p != x2 % p:
        y2 = int(crt([int(Q[1]), y1], [p, abs(x2 - x1)]))
        # y2 = int(crt([Q[1], -y1], [p, abs(x2-x1)]))

        A1 = (y2**2 - y1**2) // (x2 - x1) - (x2**3 - x1**3) // (x2 - x1)
    else:
        x2 = x1
        A1 = int(A) + p * randint(0, p)

        if Q == P:
            y2 = y1
        else:
            y2 = -y1

    B1 = y1**2 - x1**3 - A1 * x1

    assert y1**2 == x1**3 + A1 * x1 + B1
    assert y2**2 == x2**3 + A1 * x2 + B1

    return A1, B1, (x1, y1), (x2, y2)

def solve_dlp_p2(A, B, P, Q, p):
    A1, B1, P1, Q1 = lift(A, B, P, Q, p)
    x1, y1 = P1
    x2, y2 = Q1

    E1 = EllipticCurve(Zmod(p**2), [A1, B1])
    P2 = (p - 1) * E1(P1)
    Q2 = (p - 1) * E1(Q1)

    x3, y3 = ZZ(P2[0]), ZZ(P2[1])
    x4, y4 = ZZ(Q2[0]), ZZ(Q2[1])

    if x3 == x1 or x4 == x2:
        return None

    m1 = p * (y3 - y1) / (x3 - x1)  # or just m1 / m2 without *p
    m2 = p * (y4 - y2) / (x4 - x2)
    if gcd(m1.denominator(), p) != 1 or gcd(m2.denominator(), p) != 1:
        return None

    return (m1 / m2) % p

$\psi(n) = x^{n^2 / 2} + ... $

$\phi(n)$

$\omega(n)$

$n * (x, y) = (\phi_n(x) / \psi_n(x)^2, \omega_n(x) / \psi_n(x)^3 * y)$

In [16]:
res = solve_dlp_p2(a, b, P, Q, p)

In [17]:
assert res * P == Q
res

6580423973451703047762062349683967852989048968020199948724694186030410852629469815861811082547699222101664953208677803837179393306680320657110903129004768

$y^2 = x^3$


$y^2 = x^2 * (x + a)$

$x^2 = a \pmod{p}$

$a^{\frac{p - 1}{2}} = 1, -1, 0$

$x^{p -1} = 1 = a^{\frac{p-1}{2}}$

In [18]:
g = GF(101)
f = g.random_element()

In [20]:
pow(f, 50)

100

In [21]:
f

74

In [22]:
legendre_symbol(74, 101)

-1